In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import io
import random
import math
import scipy.stats
import matplotlib.pylab as plt
import seaborn as sns

C:\Users\devin\AppData\Local\Temp\ipykernel_15840\4169296045.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Initialize year variable
year = 2025 #(2019, 2021, 2022, 2023, 2024, 2025)

In [3]:
# Read in data based on the year
if year == 2019:
    data = pd.read_csv('data/18_19tourneycs.csv')
elif year == 2021:
    data = pd.read_csv('data/20_21tourneycs.csv')
elif year == 2022:
    data = pd.read_csv('data/21_22tourneycs.csv')
elif year == 2023:
    data = pd.read_csv('data/22_23tourneycs.csv')
elif year == 2024:
    data = pd.read_csv('data/23_24tourneycs.csv')
elif year == 2025:
    data = pd.read_csv('data/24_25tourneycs.csv')
data.head()

,Region,Seed,School,SRS
0,1,1,Auburn,29.58
1,1,16,Alabama State,-9.22
2,1,8,Louisville,19.30
3,1,9,Creighton,16.36
4,1,4,Texas A&M,20.05


In [4]:
# Rename columns of data DataFrame
data.columns = ['Region', 'Seed', 'Team', 'SRS']
data

,Region,Seed,Team,SRS
0,1,1,Auburn,29.58
1,1,16,Alabama State,-9.22
2,1,8,Louisville,19.30
3,1,9,Creighton,16.36
4,1,4,Texas A&M,20.05
...,...,...,...,...
59,4,11,Xavier,15.57
60,4,7,UCLA,19.24
61,4,10,Utah State,14.48
62,4,2,Tennessee,25.04


In [5]:
num_of_sims = 1000  # Number of simulations
teams = data.to_numpy().tolist()  # Convert DataFrame to a nested list
team_dict = {}  # Dictionary to store team data

# Iterate through each team
for i in range(0, len(teams)):
    # Initialize an empty list for each team in the team_dict
    team_dict[teams[i][2]] = [0, 0, 0, 0, 0, 0, 0]
    # The team name is accessed using teams[i][2], assuming the team name is in the third column of the data

In [6]:
"""
Given a list of teams in a specific round and the round number,
find_winners randomly determines the winners of each matchup based on the
SRS difference between the teams and returns a list of the winners
"""
def find_winners(input_list, round):

    winners = []

    # Iterate over the input list of teams
    for i in range(0, len(input_list)):
        # Skips if the index is odd (represents the second team in a matchup)
        if i % 2 != 0:
            pass
        else:
            # Get the first team and the second team in the matchup
            team1 = input_list[i]
            team2 = input_list[i+1]

            # Calculate the SRS difference between the teams
            srs_diff = team1[3] - team2[3]

            # Calculate the win probability of Team 1 using the SRS difference
            team1_prob = scipy.stats.norm(0, 10.36).cdf(srs_diff)

            # Generate a random draw between 0 and 1
            draw = random.random()

            # Determine the winner based on the draw and win probability
            if draw <= team1_prob:
                # Team 1 wins
                winners.append(team1)
                team_dict[team1[2]][round] += 1
            else:
                # Team 2 wins
                winners.append(team2)
                team_dict[team2[2]][round] += 1

    return winners


In [7]:
"""
Prints out the bracket representation of the tournament rounds
based on the provided round_list
"""
def bracket_printout(round_list):

    for i in range(0, len(round_list)):
        temp_list = round_list[i]

        # Iterate over the matchups in the current round
        for j in range(0, len(temp_list)):
            # Print the team seed, name, and SRS for each matchup
            print(str(temp_list[j][1]) + " " + temp_list[j][2] + " " + str(temp_list[j][3]))

            # Print an empty line after every even-indexed matchup
            if j % 2 != 0:
                print("")

        # Print empty lines between each round
        print("")
        print("")
        print("")


In [8]:
"""
Simulates a tournament by finding winners in each round and printing the bracket
"""
def tourney_sim(teams):
    # Increment the count of tournaments for each team
    for x in team_dict:
        team_dict[x][0] += 1

    # Find winners in the Round of 64
    round_32 = find_winners(teams, 1)

    # Find winners in the Round of 32
    sweet_16 = find_winners(round_32, 2)

    # Find winners in the Sweet 16
    elite_8 = find_winners(sweet_16, 3)

    # Find winners in the Elite 8
    final_4 = find_winners(elite_8, 4)

    # Find winners in the Final 4
    championship = find_winners(final_4, 5)

    # Find the winner in the Championship
    winner = find_winners(championship, 6)

    # Print the bracket for all rounds
    # Uncomment below to see bracket format of simulation. Heads up: it is a lot of output!
    #bracket_printout([teams, round_32, sweet_16, elite_8, final_4, championship, winner])


In [9]:
%%time
# Run the monte-carlo tournament simulation
for i in range(1, num_of_sims+1):
    tourney_sim(teams)

# Calculate the average statistics for each team
for x in team_dict:
    # Divide the count of wins in each round by the total number of simulations
    for i in range(0, len(team_dict[x])):
        team_dict[x][i] = team_dict[x][i] / num_of_sims

    # Print the team name and its average statistics
    print(x + " " + str(team_dict[x]))


Auburn [1.0, 1.0, 0.855, 0.715, 0.55, 0.379, 0.236]
Alabama State [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Louisville [1.0, 0.592, 0.099, 0.049, 0.016, 0.008, 0.004]
Creighton [1.0, 0.408, 0.046, 0.021, 0.007, 0.002, 0.0]
Texas A&M [1.0, 0.902, 0.485, 0.11, 0.048, 0.02, 0.007]
Yale [1.0, 0.098, 0.018, 0.0, 0.0, 0.0, 0.0]
Michigan [1.0, 0.741, 0.418, 0.098, 0.044, 0.018, 0.003]
UC San Diego [1.0, 0.259, 0.079, 0.007, 0.002, 0.0, 0.0]
Iowa State [1.0, 0.959, 0.672, 0.394, 0.147, 0.064, 0.024]
Lipscomb [1.0, 0.041, 0.003, 0.0, 0.0, 0.0, 0.0]
Mississippi [1.0, 0.538, 0.177, 0.06, 0.013, 0.002, 0.001]
North Carolina [1.0, 0.462, 0.148, 0.049, 0.01, 0.002, 0.0]
Marquette [1.0, 0.636, 0.231, 0.105, 0.027, 0.005, 0.0]
New Mexico [1.0, 0.364, 0.092, 0.022, 0.001, 0.0, 0.0]
Michigan State [1.0, 0.993, 0.677, 0.37, 0.135, 0.059, 0.018]
Bryant [1.0, 0.007, 0.0, 0.0, 0.0, 0.0, 0.0]
Florida [1.0, 0.997, 0.856, 0.615, 0.441, 0.249, 0.12]
Norfolk State [1.0, 0.003, 0.0, 0.0, 0.0, 0.0, 0.0]
Connecticut [1.0

In [18]:
expected_games = {}

for team, probs in team_dict.items():
    # Sum only the first six probabilities
    expected_games[team] = sum(probs[:6])

# Print results
for team, exp in expected_games.items():
    print(f"{team}: {exp:.3f}")

Auburn: 4.499
Alabama State: 1.000
Louisville: 1.764
Creighton: 1.484
Texas A&M: 2.565
Yale: 1.116
Michigan: 2.319
UC San Diego: 1.347
Iowa State: 3.236
Lipscomb: 1.044
Mississippi: 1.790
North Carolina: 1.671
Marquette: 2.004
New Mexico: 1.479
Michigan State: 3.234
Bryant: 1.007
Florida: 4.158
Norfolk State: 1.003
Connecticut: 1.715
Oklahoma: 1.501
Maryland: 3.282
Grand Canyon: 1.043
Memphis: 1.524
Colorado State: 1.717
Texas Tech: 3.256
UNC Wilmington: 1.017
Missouri: 2.429
Drake: 1.153
Kansas: 2.136
Arkansas: 1.502
St. John's: 2.994
Omaha: 1.011
Duke: 4.423
American: 1.000
Mississippi State: 1.488
Baylor: 1.720
Arizona: 3.085
Akron: 1.029
Oregon: 2.114
Liberty: 1.207
Wisconsin: 2.877
Montana: 1.024
BYU: 2.112
VCU: 1.467
Saint Mary's: 1.790
Vanderbilt: 1.472
Alabama: 3.783
Robert Morris: 1.002
Houston: 3.905
SIU Edwardsville: 1.000
Gonzaga: 2.234
Georgia: 1.329
Purdue: 2.837
High Point: 1.059
Clemson: 2.288
McNeese State: 1.204
Kentucky: 2.828
Troy: 1.039
Illinois: 2.338
Xavier: 1.39

In [10]:
point_system = [0, 1, 2, 3, 5, 8, 13]  # Predefined point system for each round

expected_values = {}  # Dictionary to store the expected value for each team

# Calculate the expected value for each team
for team, probabilities in team_dict.items():
    expected_value = sum(probabilities[i] * point_system[i] for i in range(len(probabilities)))
    expected_values[team] = expected_value

# Sort teams by expected value in descending order
sorted_teams = sorted(expected_values.items(), key=lambda x: x[1], reverse=True)

# Print the expected value for each team
for team, expected_value in sorted_teams:
    print(f"{team}: {expected_value:.3f}")

Auburn: 13.705
Duke: 13.064
Florida: 10.311
Houston: 9.501
Alabama: 7.598
Tennessee: 6.674
Texas Tech: 5.179
Iowa State: 5.044
Maryland: 4.995
Michigan State: 4.838
Arizona: 4.374
St. John's: 4.013
Kentucky: 3.443
Wisconsin: 3.432
Purdue: 3.313
Texas A&M: 2.693
Illinois: 2.479
Missouri: 2.423
Gonzaga: 2.342
Michigan: 2.274
Kansas: 2.040
Clemson: 1.942
BYU: 1.755
Marquette: 1.588
Oregon: 1.514
UCLA: 1.399
Mississippi: 1.166
Louisville: 1.133
Saint Mary's: 1.086
Baylor: 0.987
North Carolina: 0.971
Connecticut: 0.967
Colorado State: 0.890
Arkansas: 0.694
VCU: 0.630
Memphis: 0.625
New Mexico: 0.619
Creighton: 0.614
Oklahoma: 0.600
Mississippi State: 0.582
Vanderbilt: 0.574
Xavier: 0.557
UC San Diego: 0.448
Utah State: 0.429
Georgia: 0.419
McNeese State: 0.241
Liberty: 0.225
Drake: 0.166
Yale: 0.134
High Point: 0.067
Grand Canyon: 0.048
Lipscomb: 0.047
Troy: 0.041
Akron: 0.031
Montana: 0.024
UNC Wilmington: 0.019
Omaha: 0.012
Wofford: 0.009
Bryant: 0.007
Norfolk State: 0.003
Robert Morris: 

In [23]:
# Convert dictionary to DataFrame
df_expected_values = pd.DataFrame.from_dict(expected_values, orient='index', columns=['Expected Value'])

# Sort teams by expected value in descending order
df_expected_values = df_expected_values.sort_values(by='Expected Value', ascending=False)

# Reset the index
df_expected_values = df_expected_values.reset_index().rename(columns={'index': 'Team'})

# Display the DataFrame
display(df_expected_values)

,Team,Expected Value
0,Auburn,13.705
1,Duke,13.064
2,Florida,10.311
3,Houston,9.501
4,Alabama,7.598
...,...,...
59,Norfolk State,0.003
60,Robert Morris,0.002
61,SIU Edwardsville,0.000
62,Alabama State,0.000


In [12]:
# Merge the dataframes based on the 'Team' column
df_merged = data.merge(df_expected_values, on='Team')

# Display the merged DataFrame
df_merged.head()

,Region,Seed,Team,SRS,Expected Value
0,1,1,Auburn,29.58,13.705
1,1,16,Alabama State,-9.22,0.000
2,1,8,Louisville,19.30,1.133
3,1,9,Creighton,16.36,0.614
4,1,4,Texas A&M,20.05,2.693


In [13]:
# Define the mapping of seed values to cost values
seed_cost_mapping = {
    1: 75,
    2: 40,
    3: 25,
    4: 20,
    5: 17,
    6: 15,
    7: 12,
    8: 10,
    9: 9,
    10: 8,
    11: 7,
    12: 6,
    13: 5,
    14: 4,
    15: 3,
    16: 1,
}

# Add a new column 'Cost' to df_merged based on the 'Seed' column
df_merged['Cost'] = df_merged['Seed'].map(seed_cost_mapping)

# Display the merged DataFrame
df_merged.head()

,Region,Seed,Team,SRS,Expected Value,Cost
0,1,1,Auburn,29.58,13.705,75
1,1,16,Alabama State,-9.22,0.000,1
2,1,8,Louisville,19.30,1.133,10
3,1,9,Creighton,16.36,0.614,9
4,1,4,Texas A&M,20.05,2.693,20


In [14]:
# Define the knapsack function
def knapsack(items, budget):
    n = len(items)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, budget + 1):
            cost = items[i - 1]['Cost']
            value = items[i - 1]['Expected Value']
            if cost > j:
                dp[i][j] = dp[i - 1][j]
            else:
                dp[i][j] = max(dp[i - 1][j], value + dp[i - 1][j - cost])

    selected = []
    j = budget
    for i in range(n, 0, -1):
        if dp[i][j] != dp[i - 1][j]:
            selected.append(items[i - 1])
            j -= items[i - 1]['Cost']

    return selected

# Run the knapsack optimization on df_merged
selected_teams = knapsack(df_merged.to_dict('records'), budget=224)

# Create a new DataFrame for the selected teams
df_selected = pd.DataFrame(selected_teams)

# Display the selected teams and their information
display(df_selected)

# Calculate the total sum of the cost and expected value columns
total_cost = df_selected['Cost'].sum()
total_expected_value = df_selected['Expected Value'].sum()

# Print the total sums
print(f"Total Cost: {total_cost}")
print(f"Total Expected Value: {total_expected_value:.3f}")

,Region,Seed,Team,SRS,Expected Value,Cost
0,4,8,Gonzaga,23.63,2.342,10
1,3,2,Alabama,25.97,7.598,40
2,3,4,Arizona,23.14,4.374,20
3,3,9,Baylor,19.43,0.987,9
4,2,3,Texas Tech,23.91,5.179,25
5,2,4,Maryland,23.54,4.995,20
6,1,3,Iowa State,23.05,5.044,25
7,1,1,Auburn,29.58,13.705,75


Total Cost: 224
Total Expected Value: 44.224
